<a href="https://colab.research.google.com/github/ambwhl/datasci_223/blob/Final-Project/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Final Project: Kidney Renal Clear Cell Carcinoma (KIRC) Survival prediction and biomarker identification
Environment: the whole process in this jupiterbook was conducted on google colab.
Description below also can be found in Readme:https://github.com/ambwhl/datasci_223/edit/Final-Project/README.md

### Introduction
I am intersected in building prognostic prediction model and finding the potential biomarker for cancer patient. The data I used for this project is a subset of kidney renal clear cell carcinoma (KIRC), containing of 243 patients (subjects) and 16380 variables. The variables (predictors) are very rich: clinical covariates (e.g. cancer stage, tumor grade, and survival status), messenger RNA (mRNA) expression, microRNA (miRNA) expression, and copy number variation (CNV).
### Strategy and methods
1. Dataset was downloaded, combined, and made subset from this public project: https://www.synapse.org/#!Synapse:syn1710282/wiki/27303. Preprocessed data was saved in google drive for further analysis: https://drive.google.com/file/d/1TX0MHLrz51wpUBCalsTpivVnBSFBvIGJ/view?usp=drive_link
2. Upon initial data checking, I found several molecular variables with 0 value across all subjects. There variables were considered providing no information hence removed. After cleaning, 15882 variables remained.
3. Data set was split to train and test set, 70% in train and 30” in test. Survival status of subject in 'OS_vital_status' column was extracted as outcome.
4. 6 basic models Gradient Boosting, Neural Network, SVM, Decision Tree, Random Forest and kNN were trained and cross-validated in 5- and 10-fold manner. Performance of each model was evaluated as AUC, accuracy, F1-score, precision, and recall. Results shows that Gradient Boosting, SVM and Random Forest have the best performance in both 5-fold and 10-fold cross-validation. These 3 models were used in stacking model for a better prediction power in the next step.
5. Top3 models: Gradient Boosting, SVM and Random Forest were stacked and tested its prediction power in test data set. logistic regression was used as the final estimator. The AUC and other performance indicator were greatly improved by stacking model as compared to the basic model: AUC enhanced to 0.813, suggesting prediction power was improved.
6. Feature importance generated by the model indicates how crucial a feature (variable) is to the model. As SVM does give out feature importance directly, only feature importance from Gradient Boosting and Random Forest were used to pin down the top 10 variables as potential prognostic biomarkers. It must be noticed that weight of different basic models is difficult and take much careful consideration, so here I just average the feature importance of these 2 models. The top 1 variable with the highest importance is mRNA_PSRC1|84722.
###Discussion
The top variable is  is a message RNA of gene PSRC1. PSRC1 expressed in several cancers. Notably, it has been established as a prognostic marker in renal cancer (unfavorable) and liver cancer (unfavorable)(Ref.1). The variable ranks the 2nd, which is message RNA of Gene TPX2, is also considered as a prognostic indicator and potential therapeutic target in clear cell renal cell carcinoma (Ref.2). I would say this stacking model, eventhough confined to its samlle sample size (243) and limited population, still shows improved prediction power and strong biomarker identification capacity.
###Reference
Ref1.https://www.proteinatlas.org/ENSG00000134222-PSRC1/pathology
Ref2.https://pubmed.ncbi.nlm.nih.gov/28108243/

In [3]:
# Uncomment and install below packages if not already installed
#%pip install -q numpy pandas scikit-learn

In [ ]:
## environment requirement
%reset -f
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


In [25]:
#1.Read in preprocessed KIRC survival data
file_id = '1TX0MHLrz51wpUBCalsTpivVnBSFBvIGJ'
link = f'https://drive.google.com/uc?export=download&id={file_id}'
data = pd.read_csv(link)
print('subject number:',  len(data.index))
print('variable number:',  len(data.columns) - 2) ##minus the "feature" (subject index) and 'OS_vital_status' (survival outcome) columns.
print('example of variables:', data.columns[1:11])


subject number: 243
variable number: 16380
example of variables: Index(['age', 'gender', 'grade', 'stage', 'OS_vital_status', 'CNV_20q',
       'CNV_20p', 'CNV_Xq11.2', 'CNV_14q', 'CNV_17q24.3'],
      dtype='object')


In [28]:
##2. Remove variables with all 0 value
missindex=data.columns[(data==0).all()]
df= data[missindex]
#print(len(df.columns))
newdata= data.drop(missindex,axis=1)

#print(newdata.columns)
newdata = newdata.set_index('feature')
for col in ['gender', 'grade', 'stage']:
    newdata[col] = newdata[col].astype('category')
print("variable numbers after cleaning:",len(newdata.columns)-1)

variable numbers after cleaning: 15882


In [6]:
## 3. Split to train and test data
from sklearn.model_selection import train_test_split
X = newdata.drop(columns=['OS_vital_status'])
Y = newdata['OS_vital_status']
#print(len(Y))
#print(len(X.columns))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [30]:
## 4. Train and cross-validate basic models
# Basic models
classifiers = {
    'Gradient Boosting': GradientBoostingClassifier(),
    #'Logistic Regression': LogisticRegression(),
    'Neural Network': MLPClassifier(),
    'SVM': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'kNN': KNeighborsClassifier()
}

# performance indicator
scoring = {
    'AUC': 'roc_auc',
    'Accuracy': 'accuracy',
    'F1 Score': 'f1',
    'Precision': 'precision',
    'Recall': 'recall'
}

# 5-fold cross-validation and performance indicator for each model
print("5-fold cross-validation")
for clf_name, clf in classifiers.items():
    print(f"Evaluation metrics for {clf_name}:")
    for metric_name, scoring_method in scoring.items():
        scores = cross_val_score(clf, X_train, Y_train, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring=scoring_method)
        print(f"{metric_name}: {np.mean(scores):.4f} (std: {np.std(scores):.4f})")
    print("\n")

print("\n")
# 10-fold cross-validation and performance indicator for each model
print("10-fold cross-validation")
for clf_name, clf in classifiers.items():
    print(f"Evaluation metrics for {clf_name}:")
    for metric_name, scoring_method in scoring.items():
        scores = cross_val_score(clf, X_train, Y_train, cv=KFold(n_splits=10, shuffle=True, random_state=42), scoring=scoring_method)
        print(f"{metric_name}: {np.mean(scores):.4f} (std: {np.std(scores):.4f})")
    print("\n")


5-fold cross-validation
Evaluation metrics for Gradient Boosting:
AUC: 0.7739 (std: 0.0791)
Accuracy: 0.7647 (std: 0.0617)
F1 Score: 0.6313 (std: 0.0828)
Precision: 0.7371 (std: 0.1413)
Recall: 0.6238 (std: 0.1560)


Evaluation metrics for Neural Network:
AUC: 0.6218 (std: 0.1434)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy: 0.6412 (std: 0.1267)
F1 Score: 0.4353 (std: 0.1722)
Precision: 0.5287 (std: 0.1667)
Recall: 0.4067 (std: 0.3762)


Evaluation metrics for SVM:
AUC: 0.6819 (std: 0.0637)
Accuracy: 0.6529 (std: 0.0506)
F1 Score: 0.2912 (std: 0.1357)
Precision: 0.7309 (std: 0.3393)
Recall: 0.2195 (std: 0.1525)


Evaluation metrics for Decision Tree:
AUC: 0.6076 (std: 0.0830)
Accuracy: 0.6529 (std: 0.0900)
F1 Score: 0.5374 (std: 0.1219)
Precision: 0.5220 (std: 0.1266)
Recall: 0.5264 (std: 0.1044)


Evaluation metrics for Random Forest:
AUC: 0.7550 (std: 0.0913)
Accuracy: 0.7647 (std: 0.0671)
F1 Score: 0.5931 (std: 0.1463)
Precision: 0.7300 (std: 0.1600)
Recall: 0.5451 (std: 0.1981)


Evaluation metrics for kNN:
AUC: 0.6097 (std: 0.1224)
Accuracy: 0.6412 (std: 0.0681)
F1 Score: 0.3193 (std: 0.1763)
Precision: 0.5413 (std: 0.3495)
Recall: 0.2469 (std: 0.1453)




10-fold cross-validation
Evaluation metrics for Gradient Boosting:
AUC: 0.7469 (std: 0.1178)
Accuracy: 0.7294 (std: 0.1372)
F1 Score: 0.5

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.3265 (std: 0.2144)
Recall: 0.4203 (std: 0.3692)


Evaluation metrics for SVM:
AUC: 0.7010 (std: 0.1111)
Accuracy: 0.6529 (std: 0.1160)
F1 Score: 0.2909 (std: 0.2105)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.5875 (std: 0.3955)
Recall: 0.2379 (std: 0.2103)


Evaluation metrics for Decision Tree:
AUC: 0.6084 (std: 0.1455)
Accuracy: 0.6471 (std: 0.1488)
F1 Score: 0.5071 (std: 0.1544)
Precision: 0.5227 (std: 0.2391)
Recall: 0.4899 (std: 0.1994)


Evaluation metrics for Random Forest:
AUC: 0.7786 (std: 0.1217)
Accuracy: 0.7706 (std: 0.1034)
F1 Score: 0.6561 (std: 0.1142)
Precision: 0.8505 (std: 0.2065)
Recall: 0.6085 (std: 0.1771)


Evaluation metrics for kNN:
AUC: 0.5958 (std: 0.1113)
Accuracy: 0.6294 (std: 0.1086)
F1 Score: 0.2963 (std: 0.1986)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.4917 (std: 0.3698)
Recall: 0.2419 (std: 0.1785)




In [8]:
## 5. Stack the models with top3 performance
# top 3 models
gradient_boosting = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
svm = make_pipeline(StandardScaler(), SVC(random_state=0))
random_forest = RandomForestClassifier(n_estimators=100, random_state=0)

# Stacking
stacked_model = StackingClassifier(
    estimators=[
        ('gradient_boosting', gradient_boosting),
        ('svm', svm),
        ('random_forest', random_forest)
    ],
    final_estimator=LogisticRegression()
)

stacked_model.fit(X_train, Y_train)
predictions = stacked_model.predict(X_test)

y_proba = stacked_model.predict_proba(X_test)[:, 1]
y_pred = stacked_model.predict(X_test)

#  performance indicator
auc = roc_auc_score(Y_test, y_proba)
accuracy = accuracy_score(Y_test, y_pred)
f1 = f1_score(Y_test, y_pred)
precision = precision_score(Y_test, y_pred)
recall = recall_score(Y_test, y_pred)

metrics_stacked = {
     'AUC': auc,
     'Accuracy': accuracy,
     'F1 Score': f1,
     'Precision': precision,
     'Recall': recall
 }
print(metrics_stacked)

{'AUC': 0.8126984126984127, 'Accuracy': 0.7397260273972602, 'F1 Score': 0.5777777777777777, 'Precision': 0.7647058823529411, 'Recall': 0.4642857142857143}


In [16]:
## 6. Pin down biomarkers with top feature importance
predictors = X.columns.tolist()

#feature importance
gb_imp = stacked_model.named_estimators_['gradient_boosting'].feature_importances_
rf_imp = stacked_model.named_estimators_['random_forest'].feature_importances_

# Average the feature importances as weight
imp = (gb_imp + rf_imp) / 2

# top 10 variables
topind = np.argsort(imp)[::-1][:10]
top = [(predictors[i], imp[i]) for i in topind]

##show the top variables and their importance, these are the potentail biomarker
for feature, importance in top:
    print(f"{feature}: {importance}")


mRNA_PSRC1|84722: 0.16786182986701073
mRNA_TPX2|22974: 0.05760733865569538
stage: 0.05708578005415763
mRNA_NCOR1|9611: 0.037316728895259416
mRNA_HK3|3101: 0.02725911232028246
mRNA_PMPCA|23203: 0.025781179652827047
mRNA_CYP26B1|56603: 0.02566115735037458
mRNA_CITED2|10370: 0.02443062193144756
miRNA_hsa-mir-335: 0.016659761568937028
mRNA_FAM177B|400823: 0.013477700423530253
